# Análisis de código fuente de repositorios (líneas de código)
Este análisis netamente considera las líneas de código (LOC) de cada repositorio.

## Cargar dependencias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Cargar archivo CSV de entrada

In [2]:
# generamos DF a partir de CSV de entrada
input_file = '../csvs/cloc_output.csv'
raw_df = pd.read_csv(input_file)
raw_df

,project_id,language,files,blank,comment,code
0,0xEduardo_nodeless,JSON,2,0,0,1493
1,0xEduardo_nodeless,Markdown,1,41,0,75
2,0xEduardo_nodeless,JavaScript,1,7,0,34
3,0xEduardo_nodeless,YAML,1,5,0,32
4,0xEduardo_nodeless,SUM,5,53,0,1634
...,...,...,...,...,...,...
5409,zotoio_github-task-manager,Bourne,Shell,2,6,2
5410,zotoio_github-task-manager,Dockerfile,1,4,0,12
5411,zotoio_github-task-manager,SVG,1,0,0,1
5412,zotoio_github-task-manager,Text,1,0,0,1


Copiamos este DataFrame para convertir el tipo de dato de las columnas a numérico

In [3]:
df = raw_df.copy()
df['files'] = pd.to_numeric(df['files'], errors='coerce')
df['blank'] = pd.to_numeric(df['blank'], errors='coerce')
df['comment'] = pd.to_numeric(df['comment'], errors='coerce')
df['code'] = pd.to_numeric(df['code'], errors='coerce')

In [4]:
# guardamos el nuevo DataFrame en un nuevo CSV
output_file = '../temp_data/merged_code_analysis.csv'
df.to_csv(output_file, index=False)

## Análisis estadístico

In [5]:
filtered_df = df[df['language'] != 'SUM']
filtered_df

,project_id,language,files,blank,comment,code
0,0xEduardo_nodeless,JSON,2.0,0.0,0,1493
1,0xEduardo_nodeless,Markdown,1.0,41.0,0,75
2,0xEduardo_nodeless,JavaScript,1.0,7.0,0,34
3,0xEduardo_nodeless,YAML,1.0,5.0,0,32
5,0xayot_waas,JSON,3.0,0.0,0,32446
...,...,...,...,...,...,...
5408,zotoio_github-task-manager,XML,1.0,0.0,0,17
5409,zotoio_github-task-manager,Bourne,NaN,2.0,6,2
5410,zotoio_github-task-manager,Dockerfile,1.0,4.0,0,12
5411,zotoio_github-task-manager,SVG,1.0,0.0,0,1


In [6]:
# group by 'language' and aggregate the sum of columns
sums_per_language = filtered_df.groupby('language').agg({
    'files': 'sum',
    'code': 'sum'
}).reset_index()
# calculate the percentage of each language based on the 'code' column
sums_per_language['percentage'] = (sums_per_language['code'] / sums_per_language['code'].sum()) * 100
sums_per_language
sums_per_language.sort_values(by='code', ascending=False, inplace=True)
sums_per_language.head(20)

,language,files,code,percentage
43,JavaScript,55174.0,14998967,31.072053
38,JSON,24611.0,14838233,30.739075
59,PHP,46265.0,7320612,15.165474
85,TypeScript,38223.0,3473117,7.194954
13,CSS,12969.0,2600957,5.388176
74,SQL,406.0,1751122,3.627647
51,Markdown,8012.0,969358,2.008135
14,CSV,118.0,689432,1.428236
93,YAML,3509.0,261789,0.542325
75,SVG,4176.0,188679,0.390870


In [7]:
keep_languages = ["JavaScript", "TypeScript", "C#", "PowerShell", "Ruby", "Java", "Python", "Go"]
mask = ~sums_per_language["language"].isin(keep_languages)
sums_per_language.loc[mask, "language"] = "Other"
sums_per_language = sums_per_language.groupby("language", as_index=False).agg({"files": "sum", "code": "sum"})
sums_per_language = sums_per_language.sort_values(by='code', ascending=False)
sums_per_language

,language,files,code
4,Other,108448.0,29515656
3,JavaScript,55174.0,14998967
8,TypeScript,38223.0,3473117
6,Python,2116.0,185582
1,Go,704.0,58050
0,C#,272.0,19892
2,Java,345.0,18962
7,Ruby,38.0,1181
5,PowerShell,2.0,160
